In [49]:
import tkinter as tk
from tkinter import messagebox, ttk
import csv
import random

In [ ]:
from tkinter import simpledialog, messagebox
import tkinter as tk
import csv

# Global variable to hold movie data
movies = []

def load_movies_from_csv(file_path):
    global movies
    movies = []
    try:
        with open(file_path, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                movies.append({
                    "title": row['title'],
                    "genre": row['genre'],
                    "cast_quality": row['cast_quality'],
                    "production": row['production'],
                    "budget": row['budget'],
                    "box_office": row['box_office'],
                    "critical_acclaim": float(row['critical_acclaim'])
                })
        movies.sort(key=lambda x: x['title'])  # Ensure movies are sorted by title for binary search
    except Exception as e:
        print(f"Error reading CSV file: {e}")

def add_movie(title, genre, cast_quality, production, budget, box_office, critical_acclaim):
    new_movie = {
        "title": title,
        "genre": genre,
        "cast_quality": cast_quality,
        "production": production,
        "budget": budget,
        "box_office": box_office,
        "critical_acclaim": float(critical_acclaim)
    }
    movies.append(new_movie)
    movies.sort(key=lambda x: x['title'])

def delete_movie(title):
    global movies
    movies = [movie for movie in movies if movie['title'].lower() != title.lower()]
    messagebox.showinfo("Deleted", f"Movie '{title}' deleted successfully!")

def binary_search_movies(query, movies):
    low, high = 0, len(movies) - 1
    results = []
    while low <= high:
        mid = (low + high) // 2
        if query.lower() in movies[mid]['title'].lower():
            results.append(movies[mid])
            left, right = mid - 1, mid + 1
            while left >= 0 and query.lower() in movies[left]['title'].lower():
                results.append(movies[left])
                left -= 1
            while right < len(movies) and query.lower() in movies[right]['title'].lower():
                results.append(movies[right])
                right += 1
            break
        elif query.lower() < movies[mid]['title'].lower():
            high = mid - 1
        else:
            low = mid + 1

    for movie in movies:
        if query.lower() in movie['genre'].lower() and movie not in results:
            results.append(movie)

    return results

def merge_sort_movies(movies):
    if len(movies) > 1:
        mid = len(movies) // 2
        left_half = movies[:mid]
        right_half = movies[mid:]

        merge_sort_movies(left_half)
        merge_sort_movies(right_half)

        i, j, k = 0, 0, 0

        while i < len(left_half) and j < len(right_half):
            if left_half[i]['critical_acclaim'] > right_half[j]['critical_acclaim']:
                movies[k] = left_half[i]
                i += 1
            else:
                movies[k] = right_half[j]
                j += 1
            k += 1

        while i < len(left_half):
            movies[k] = left_half[i]
            i += 1
            k += 1

        while j < len(right_half):
            movies[k] = right_half[j]
            j += 1
            k += 1

def recommend_movies(top_n, genre=None):
    if genre:
        filtered_movies = [movie for movie in movies if genre.lower() in movie['genre'].lower()]
    else:
        filtered_movies = movies[:]
    
    merge_sort_movies(filtered_movies)
    filtered_movies.reverse()  # Reverse to get descending order
    top_movies = filtered_movies[:top_n]
    for i, movie in enumerate(top_movies):
        movie['recommended_rating'] = calculate_unique_rating(movie, i)
    return top_movies

def calculate_unique_rating(movie, index):
    genre_weight = {"Sci-Fi": 2, "Drama": 1.5, "Action": 1.8, "Animation": 1.2, "Comedy": 1.1, "Horror": 1.3, "Thriller": 1.4, "Romance": 1.0}
    cast_weight = {"A-list": 2, "B-list": 1}
    production_weight = {"Warner Bros": 2, "Aamir Khan Productions": 1.8, "Arka Media Works": 1.7, "Marvel Studios": 2, "Barunson E&A": 1.6, "Disney": 1.9, "Vinod Chopra Films": 1.7, "Sony Pictures Networks": 1.5, "Zee Studios": 1.4, "GP Sippy": 1.3, "Phantom Films": 1.6, "Cine1 Studios": 1.4, "Hombale Films": 1.5, "Matchbox Pictures": 1.5, "Yash Raj Films": 1.8, "Salman Khan Films": 1.6, "Sohum Shah Films": 1.4, "Maddock Films": 1.3}
    budget_weight = {"High": 2, "Medium": 1.5, "Low": 1}
    box_office_weight = {"Blockbuster": 2, "Hit": 1.5}

    rating = (
        movie['critical_acclaim'] +
        genre_weight.get(movie['genre'], 1) +
        cast_weight.get(movie['cast_quality'], 1) +
        production_weight.get(movie['production'], 1) +
        budget_weight.get(movie['budget'], 1) +
        box_office_weight.get(movie['box_office'], 1)
    )
    
    rating = min(10, rating / 2)
    return rating - index * 0.01

def add_movie_gui():
    title = simpledialog.askstring("Input", "Enter movie title:")
    genre = simpledialog.askstring("Input", "Enter movie genre:")
    cast_quality = simpledialog.askstring("Input", "Enter cast quality (A-list/B-list):")
    production = simpledialog.askstring("Input", "Enter production company:")
    budget = simpledialog.askstring("Input", "Enter budget (Low/Medium/High):")
    box_office = simpledialog.askstring("Input", "Enter box office (Hit/Blockbuster):")
    critical_acclaim = simpledialog.askfloat("Input", "Enter critical acclaim (0-10):")

    if title and genre and cast_quality and production and budget and box_office and critical_acclaim is not None:
        add_movie(title, genre, cast_quality, production, budget, box_office, critical_acclaim)
        messagebox.showinfo("Success", f"Movie '{title}' added successfully!")
    else:
        messagebox.showerror("Error", "All fields are required!")

def delete_movie_gui():
    title = simpledialog.askstring("Input", "Enter movie title to delete:")
    if title:
        delete_movie(title)

def search_movie_gui():
    query = simpledialog.askstring("Input", "Enter movie title or genre to search:")
    if query:
        results = binary_search_movies(query, movies)
        if results:
            result_str = "\n".join([f"Title: {movie['title']}, Genre: {movie['genre']}, Cast Quality: {movie['cast_quality']}, Production: {movie['production']}, Budget: {movie['budget']}, Box Office: {movie['box_office']}, Critical Acclaim: {movie['critical_acclaim']}" for movie in results])
            messagebox.showinfo("Search Results", result_str)
        else:
            messagebox.showinfo("Search Results", "No matching movies found.")

def recommend_movies_gui():
    choice = simpledialog.askstring("Input", "Recommend movies (general) or by genre (enter 'genre')?")
    if choice and choice.lower() == "genre":
        genre = simpledialog.askstring("Input", "Enter the genre to recommend movies for:")
        if genre:
            top_n = simpledialog.askinteger("Input", "Enter the number of top movies to recommend:")
            if top_n:
                recommendations = recommend_movies(top_n, genre)
                if recommendations:
                    result_str = "\n".join([f"{movie['title']} - {movie['recommended_rating']:.2f}" for movie in recommendations])
                    messagebox.showinfo("Top Recommendations", result_str)
                else:
                    messagebox.showinfo("Top Recommendations", f"No movies found for genre '{genre}'.")
    else:
        top_n = simpledialog.askinteger("Input", "Enter the number of top movies to recommend:")
        if top_n:
            recommendations = recommend_movies(top_n)
            result_str = "\n".join([f"{movie['title']} - {movie['recommended_rating']:.2f}" for movie in recommendations])
            messagebox.showinfo("Top Recommendations", result_str)

def main():
    load_movies_from_csv("C:/Users/Tanvi Shekhar Sawant/Downloads/movie_attributes.csv")

    root = tk.Tk()
    root.title("Movie Recommendation System")

    add_button = tk.Button(root, bg="lightblue",  text="Add Movie", command=add_movie_gui)
    add_button.pack(pady=10)

    delete_button = tk.Button(root, bg="lightblue", text="Delete Movie", command=delete_movie_gui)
    delete_button.pack(pady=10)

    search_button = tk.Button(root, bg="lightblue", text="Search Movie", command=search_movie_gui)
    search_button.pack(pady=10)

    recommend_button = tk.Button(root, bg="lightblue", text="Recommend Movies", command=recommend_movies_gui)
    recommend_button.pack(pady=10)

    root.mainloop()

if __name__ == "__main__":
    main()